In [2]:
import sys
import os
import numpy as np
import scipy.stats as stats
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sb
import random as rd
import matplotlib
import warnings

In [2]:
# path1 = 'D:/UCSF_postdoc_topic/ECHO_project/Reprocessed_MSDIAL/Reprocessed_20241003_withMoNA/Area_1_2024_10_03_09_05_38_copy.csv'
#path3 = 'D:/UCSF_postdoc_topic/ECHO_project/Reprocessed_MSDIAL/Chemical List PLASTICMAP-2024-10-03.csv'
# path3 = 'D:/UCSF_postdoc_topic/ECHO_project/Reprocessed_MSDIAL/CCD-Batch-Search_2024-10-21_07_17_40.csv'
# path1 = 'D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Area_1_2024_10_22_17_50_44_neg.csv'
# path2 = 'D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/sample_meta_neg.csv'
# path5 = 'D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/sample_meta_pos.csv'
# path4 = 'D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Area_1_2024_10_22_17_13_52_pos.csv'
# path6 = 'D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/QCtest/QCtest/Area_1_2024_10_28_13_47_02.csv'

In [ ]:
##detection frequency of thoes features with hits, include the biotransformation product and perform ms1 search again. in R
##calculate standard deviations and covariance of those features among 200 serum samples
##generate the final inclusion list for ms/ms acquisition
##with the ms2, perform molecular networking to support the plastic and their biotrnasformation products

In [5]:
from tqdm import tqdm
#do ms1 search and matching for the pos peak area for the biotransformation product
#import peak area for positive after batch correction
pos_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/filtered_peak_area_for_bc_70.csv")
# pos_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/filtered_peak_area_after_bc.csv")
# neg_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/filtered_peak_area_for_bc.csv")
    #ms1 search based on the Average Rt and Average Mz

# targetmzdat = unique_transform
# path1 = 'D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic.csv'
path2 = 'D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/PlasticChem_predRT.csv'
# targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic.csv')
targetmzdat_pos = pd.read_csv(path2)
# peak_area_inquiry= neg_peak_area_200[['Average Rt(min)', 'Average Mz']]
peak_area_inquiry= pos_peak_area_200[['Average Rt(min)', 'Average Mz']]
targetmzdat_pos['mz'] = targetmzdat_pos['Monoisotopic_Mass_ready'] + 1.007825

#import the retention time predicted by Retip



# Step 2: Match filtered_df with MSDIAL result file based on monoisotopic mass within a given ppm threshold
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

ppm_threshold = 5
targetmzdat_pos['Hits_pos'] = 0
matched_rows_pos = []

for _, chem_row in tqdm(targetmzdat_pos.iterrows(), total=targetmzdat_pos.shape[0], desc="Matching compounds"):
    mw = chem_row['mz']
    matches = peak_area_inquiry[(peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold))]
    if not matches.empty:
        targetmzdat_pos.at[chem_row.name, 'Hits_pos'] = len(matches)
        for _, match_row in matches.iterrows():
            matched_rows_pos.append({**match_row, 'Matched Compound': chem_row['SMILES'], 'PREFERRED NAME': chem_row.get('PREFERRED NAME', None), 'DTXSID': chem_row.get('DTXSID', None)})

# Create a new DataFrame for matched MSDIAL result rows
matched_peak_area_df_pos = pd.DataFrame(matched_rows_pos)
targetmzdat_pos = targetmzdat_pos[targetmzdat_pos['Hits_pos']>0]
# targetmzdat_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/filtered_plastic_ms1match_list_pos.csv") #(341, 797)
# targetmzdat.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/biotransformation_product/filtered_biotransform_product_list_neg.csv") #()
# matched_peak_area_df_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/pos_mathed_plasticmap_results.csv")
targetmzdat_pos.shape
# Save the updated filtered DataFrame and matched MSDIAL DataFrame to CSV files
# filtered_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/neg_Filtered_Chemical_List_with_Hits.csv', index=False)
# matched_msdial_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/neg_Matched_MSDIAL_Results.csv', index=False)
# filtered_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/pos_Filtered_Chemical_List_with_Hits.csv', index=False)
# matched_msdial_df.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/pos_Matched_MSDIAL_Results.csv', index=False)

# Print the number of chemicals with hits from the MSDIAL result file
# num_chemicals_with_hits = filtered_df[filtered_df['Hits'] > 0].shape[0]

C:\Users\yangj\AppData\Local\Temp\ipykernel_40708\1192439945.py:10: DtypeWarning: Columns (18,19,20,23,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic.csv')
Matching compounds: 100%|██████████| 5144/5144 [00:29<00:00, 173.22it/s]


(1121, 798)

In [ ]:
#do ms1 search and matching for the pos peak area for the biotransformation product
#import peak area for positive after batch correction
# pos_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/filtered_peak_area_after_bc.csv")
neg_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/filtered_peak_area_after_bc.csv")
    #ms1 search based on the Average Rt and Average Mz

# targetmzdat = unique_transform
targetmzdat_neg = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Plastic_Chemical_List_organic.csv')
# peak_area_inquiry= neg_peak_area_200[['Average Rt(min)', 'Average Mz']]
peak_area_inquiry= neg_peak_area_200[['Average Rt(min)', 'Average Mz']]

# Step 2: Match filtered_df with MSDIAL result file based on monoisotopic mass within a given ppm threshold
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

ppm_threshold = 5
targetmzdat_neg['Hits_neg'] = 0
matched_rows = []

for _, chem_row in tqdm(targetmzdat_neg.iterrows(), total=targetmzdat_neg.shape[0], desc="Matching compounds"):
    mw = chem_row['MONOISOTOPIC_MASS']
    matches = peak_area_inquiry[(peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold))]
    if not matches.empty:
        targetmzdat_neg.at[chem_row.name, 'Hits_neg'] = len(matches)
        for _, match_row in matches.iterrows():
            matched_rows.append({**match_row, 'Matched Compound': chem_row['SMILES'], 'PREFERRED NAME': chem_row.get('PREFERRED NAME', None), 'DTXSID': chem_row.get('DTXSID', None)})


# Create a new DataFrame for matched MSDIAL result rows
matched_peak_area_df_neg = pd.DataFrame(matched_rows)
targetmzdat_neg = targetmzdat_neg[targetmzdat_neg['Hits_neg']>0]
targetmzdat_neg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/plasticmap_filtered_feature_list_neg.csv") #(358,797)
matched_peak_area_df_neg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/neg_mathed_plasticmap_results.csv")
targetmzdat_neg.shape

C:\Users\yangj\AppData\Local\Temp\ipykernel_12656\280227006.py:8: DtypeWarning: Columns (18,19,20,23,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  targetmzdat_neg = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Plastic_Chemical_List_organic.csv')
Matching compounds: 100%|██████████| 5144/5144 [00:50<00:00, 102.10it/s]


(358, 797)

In [ ]:
#combined both pos and neg for potential identification, prioritization stretagy 1
targetmzdat_combined = pd.concat([targetmzdat_pos,targetmzdat_neg])
print(targetmzdat_combined.shape)

# combined_pos_neg = targetmzdat_combined[(targetmzdat_combined['Hits_pos']>0) & (targetmzdat_combined['Hits_neg'] >0)]
combined_pos_neg = targetmzdat_combined.drop_duplicates(subset='SMILES_ready')
duplicated_posandneg = targetmzdat_combined['SMILES_ready'].value_counts()[targetmzdat_combined['SMILES_ready'].value_counts()==2].index
len(duplicated_posandneg)

#extract the peak area of the targeted detected in positive
#extract the peak area of targeted in negative
#extract the peak area of targeted in both in pos and neg

(699, 798)


88

In [ ]:
##using chemspyder to categories the targeted list?
##using the rt prediction for the prioritizaion
##using the production volume for prioritization
##using the plenternal permieability prediction for the prirorization
combined_pos_neg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/plastic_list_match_pos_and_neg.csv")
targetmzdat_combined.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/plastic_list_match_pos_or_neg.csv")

In [ ]:
filtered_df = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Plastic_Chemical_List_organic.csv')
Export_Transformation = filtered_df[['SMILES_ready']]
Export_Transformation =Export_Transformation.rename(columns={'SMILES_ready':'SMILES'})
Export1 = Export_Transformation.iloc[0:2000,]
Export2 = Export_Transformation.iloc[2000:4000,]
Export3 = Export_Transformation.iloc[4000:5144,]
Export1.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/biotransformation_product/plast_batch1_biotran.csv', index = False)
Export2.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/biotransformation_product/plast_batch2_biotran.csv', index = False)
Export3.to_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/biotransformation_product/plast_batch3_biotran.csv', index = False)

Export1.shape[0] +Export2.shape[0] +Export3.shape[0] #5144
filtered_df.shape #5144

##clean up and combined the biotransformation product
##combined with the precursor molecules for the biotransformation product.
##combined into a big list.
import pandas as pd
import os

original_suspectlst = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Plastic_Chemical_List_organic.csv')
filepath = 'D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/biotransformation_product/'
file1 = 'bath1_ouput.csv'
file2 = 'bath2_ouput.csv'
file3 = 'bath3_ouput.csv'
transform1 = pd.read_csv(os.path.join(filepath,file1))
transform2 = pd.read_csv(os.path.join(filepath,file2))
transform3 = pd.read_csv(os.path.join(filepath,file3))

transform_combined = pd.concat([transform1,transform2,transform3],axis=0)
transform_combined.shape #(25422, 23)

# (25422, 23)
# (21944,)
# (441,)
##filter duplicated row with same SMILES, filter rows with mono isotope mass around 100-1000
print(transform_combined.shape)
filtered_transform = transform_combined[(transform_combined['Major Isotope Mass'] >=100)&(transform_combined['Major Isotope Mass'] <=1000)]
unique_product = filtered_transform['SMILES'].value_counts()[filtered_transform['SMILES'].value_counts()==1].index
# filtered_transform.drop_duplicates("Major Isotope Mass", keep=False)
# print(filtered_transform.shape) #21944,23
print(unique_product.shape)
unique_precursor = filtered_transform['Precursor SMILES'].value_counts()[filtered_transform['Precursor SMILES'].value_counts()==1].index
print(unique_precursor.shape) 
unique_transform = filtered_transform[filtered_transform['SMILES'].isin(unique_product)] #23205,466
unique_transform.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/biotransformation_product/unique_biotransformation_product_combined_mass100_1000_phase1.csv", index=False )

In [ ]:
original_suspectlst = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/First100_batch/Plastic_Chemical_List_organic.csv')
filepath = 'D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/biotransformation_product/'
file4 = 'bath1_phase2_ouput.csv'
file5 = 'bath2_phase2_ouput.csv'
file6 = 'bath3_phase2_ouput.csv'
transform4 = pd.read_csv(os.path.join(filepath,file4))
transform5 = pd.read_csv(os.path.join(filepath,file5))
transform6 = pd.read_csv(os.path.join(filepath,file6))

transform_combined_p2 = pd.concat([transform4,transform5,transform6],axis=0)

print(transform_combined_p2.shape)
filtered_transform_p2 = transform_combined_p2[(transform_combined_p2['Major Isotope Mass'] >=100)&(transform_combined_p2['Major Isotope Mass'] <=1000)]
unique_product_p2 = filtered_transform_p2['SMILES'].value_counts()[filtered_transform_p2['SMILES'].value_counts()==1].index
# filtered_transform.drop_duplicates("Major Isotope Mass", keep=False)
# print(filtered_transform.shape) #21944,23
print(unique_product_p2.shape)
unique_precursor_p2 = filtered_transform_p2['Precursor SMILES'].value_counts()[filtered_transform_p2['Precursor SMILES'].value_counts()==1].index
print(unique_precursor_p2.shape) 
unique_transform_p2 = filtered_transform_p2[filtered_transform_p2['SMILES'].isin(unique_product_p2)] 
unique_transform_p2.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/biotransformation_product/unique_biotransformation_product_combined_mass100_1000_phase2.csv", index=False )

# (5474, 23)
# (5240,)
# (536,)

phase1_2_combined = pd.concat([unique_transform,unique_transform_p2])
print(phase1_2_combined.shape)
# phase1_2_combined.drop_duplicates("Precursor SMILES", keep="last")
unique_precursor_combined = phase1_2_combined['Precursor SMILES'].value_counts()[phase1_2_combined['Precursor SMILES'].value_counts()==1].index
print(unique_precursor_combined.shape) 

# (27184, 23)
# (343,)

In [83]:
phase1_2_combined.head()

,InChI,InChIKey,SMILES,Synonyms,PUBCHEM_CID,Molecular formula,Major Isotope Mass,ALogP,Lipinski_Violations,Insecticide_Likeness_Violations,...,Reaction ID,Enzyme(s),Biosystem,Precursor ID,Precursor SMILES,Precursor InChI,Precursor InChIKey,Precursor ALogP,Precursor Major Isotope Mass,Hits_pos
0,InChI=1S/C13H22O3/c1-9(14)12(15)16-11-7-5-10(6...,FPCXHNBSKIAECM-UHFFFAOYSA-N,C=C(C(=O)OC1CCC(C(C)(C)C)CC1)O,NaN,NaN,C13H22O3,226.156895,3.1799,0.0,0.0,...,BTMR1055,CYP1A2\nCYP3A4,HUMAN,NaN,C=CC(=O)OC1CCC(C(C)(C)C)CC1,InChI=1S/C13H22O2/c1-5-12(14)15-11-8-6-10(7-9-...,LAIJAUHBAWLPCO-UHFFFAOYSA-N,NaN,210.1619,0
1,InChI=1S/C13H22O3/c1-5-12(15)16-11-7-6-9(8-10(...,SHFANGIQSWVUTM-UHFFFAOYSA-N,C=CC(=O)OC1C(CC(C(C)(C)C)CC1)O,NaN,NaN,C13H22O3,226.156895,2.7761,0.0,0.0,...,BTMR1068,CYP1A2\nCYP2C8\nCYP2C9\nCYP2C19\nCYP2D6\nCYP3A4,HUMAN,NaN,C=CC(=O)OC1CCC(C(C)(C)C)CC1,InChI=1S/C13H22O2/c1-5-12(14)15-11-8-6-10(7-9-...,LAIJAUHBAWLPCO-UHFFFAOYSA-N,NaN,210.1619,0
2,InChI=1S/C13H22O3/c1-5-12(15)16-9-6-7-10(11(14...,XKHJMOQTQPIDEI-UHFFFAOYSA-N,C=CC(=O)OC1CC(C(C(C)(C)C)CC1)O,NaN,NaN,C13H22O3,226.156895,2.4498,0.0,0.0,...,BTMR1068,CYP1A2\nCYP2C8\nCYP2C9\nCYP2C19\nCYP2D6\nCYP3A4,HUMAN,NaN,C=CC(=O)OC1CCC(C(C)(C)C)CC1,InChI=1S/C13H22O2/c1-5-12(14)15-11-8-6-10(7-9-...,LAIJAUHBAWLPCO-UHFFFAOYSA-N,NaN,210.1619,0
3,"InChI=1S/C13H22O3/c1-13(2,3)9-4-6-10(7-5-9)16-...",UTLAETFBDLVAFS-UHFFFAOYSA-N,C1C(C(=O)OC2CCC(C(C)(C)C)CC2)O1,NaN,NaN,C13H22O3,226.156895,2.8426,0.0,0.0,...,BTMR0109,CYP1A2\nCYP2B6\nCYP2B6\nCYP2C8\nCYP2C9\nCYP2C1...,HUMAN,NaN,C=CC(=O)OC1CCC(C(C)(C)C)CC1,InChI=1S/C13H22O2/c1-5-12(14)15-11-8-6-10(7-9-...,LAIJAUHBAWLPCO-UHFFFAOYSA-N,NaN,210.1619,0
4,InChI=1S/C13H22O3/c1-4-12(15)16-11-7-5-10(6-8-...,YTMFLOJRIFWYDD-UHFFFAOYSA-N,C=CC(=O)OC1CCC(C(C)(C)CO)CC1,NaN,NaN,C13H22O3,226.156895,2.6523,0.0,0.0,...,BTMR1061,CYP1A2\nCYP2A6\nCYP2B6\nCYP2C8\nCYP2C9\nCYP2C1...,HUMAN,NaN,C=CC(=O)OC1CCC(C(C)(C)C)CC1,InChI=1S/C13H22O2/c1-5-12(14)15-11-8-6-10(7-9-...,LAIJAUHBAWLPCO-UHFFFAOYSA-N,NaN,210.1619,0


In [86]:
#screening: MS1 search for the biotransformation product and get the list for CFMID prediction
#prepare SMILES_ready for CFMID MSMS spectrum prediction

#do ms1 search and matching for the pos peak area for the biotransformation product
#import peak area for positive after batch correction
pos_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/filtered_peak_area_for_bc.csv")
# neg_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/filtered_peak_area_for_bc.csv")
    #ms1 search based on the Average Rt and Average Mz

transmzdat_pos = phase1_2_combined
# peak_area_inquiry= neg_peak_area_200[['Average Rt(min)', 'Average Mz']]
peak_area_inquiry= pos_peak_area_200[['Average Rt(min)', 'Average Mz']]

# Step 2: Match filtered_df with MSDIAL result file based on monoisotopic mass within a given ppm threshold
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

ppm_threshold = 5
transmzdat_pos['Hits_pos'] = 0
matched_rows_pos = []

for _, chem_row in tqdm(transmzdat_pos.iterrows(), total=transmzdat_pos.shape[0], desc="Matching compounds"):
    mw = chem_row['Major Isotope Mass']
    matches = peak_area_inquiry[(peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold))]
    if not matches.empty:
        transmzdat_pos.at[chem_row.name, 'Hits_pos'] = len(matches)
        for _, match_row in matches.iterrows():
            matched_rows_pos.append({**match_row, 'Matched Compound': chem_row['SMILES'], 'Molecular formula': chem_row.get('Molecular formula', None), 
                                 'Precursor SMILES': chem_row.get('Precursor SMILES', None), 'PUBCHEM_CID':chem_row.get('PUBCHEM_CID', None)})


# Create a new DataFrame for matched MSDIAL result rows
trans_matched_peak__pos = pd.DataFrame(matched_rows_pos)
transmzdat_pos = transmzdat_pos[transmzdat_pos['Hits_pos']>0]
# targetmzdat_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/filtered_plastic_ms1match_list_pos.csv") #(341, 797)
# targetmzdat.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/biotransformation_product/filtered_biotransform_product_list_neg.csv") #()
# matched_peak_area_df_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/pos_mathed_plasticmap_results.csv")
transmzdat_pos.shape

Matching compounds: 100%|██████████| 27184/27184 [04:16<00:00, 106.12it/s]


(7715, 24)

In [87]:
# pos_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/filtered_peak_area_for_bc.csv")
neg_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/filtered_peak_area_for_bc.csv")
    #ms1 search based on the Average Rt and Average Mz

transmzdat_neg = phase1_2_combined
peak_area_inquiry= neg_peak_area_200[['Average Rt(min)', 'Average Mz']]
# peak_area_inquiry= pos_peak_area_200[['Average Rt(min)', 'Average Mz']]

# Step 2: Match filtered_df with MSDIAL result file based on monoisotopic mass within a given ppm threshold
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

ppm_threshold = 5
transmzdat_neg['Hits_pos'] = 0
matched_rows_neg = []

for _, chem_row in tqdm(transmzdat_neg.iterrows(), total=transmzdat_neg.shape[0], desc="Matching compounds"):
    mw = chem_row['Major Isotope Mass']
    matches = peak_area_inquiry[(peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold))]
    if not matches.empty:
        transmzdat_neg.at[chem_row.name, 'Hits_pos'] = len(matches)
        for _, match_row in matches.iterrows():
            matched_rows_neg.append({**match_row, 'Matched Compound': chem_row['SMILES'], 'Molecular formula': chem_row.get('Molecular formula', None), 
                                 'Precursor SMILES': chem_row.get('Precursor SMILES', None), 'PUBCHEM_CID':chem_row.get('PUBCHEM_CID', None)})


# Create a new DataFrame for matched MSDIAL result rows
trans_matched_peak__neg = pd.DataFrame(matched_rows_neg)
transmzdat_neg = transmzdat_neg[transmzdat_neg['Hits_pos']>0]
# targetmzdat_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/filtered_plastic_ms1match_list_pos.csv") #(341, 797)
# targetmzdat.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/biotransformation_product/filtered_biotransform_product_list_neg.csv") #()
# matched_peak_area_df_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/pos_mathed_plasticmap_results.csv")
transmzdat_neg.shape

Matching compounds: 100%|██████████| 27184/27184 [03:01<00:00, 149.43it/s]


(7573, 24)

In [11]:
import pandas as pd
from tqdm import tqdm

# Load peak area data for positive ion mode after batch correction
pos_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/filtered_peak_area_for_bc_70.csv")

# Load target mz data
targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic.csv')
#targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic_withRTprediction.csv')
targetmzdat_pos['mz'] = targetmzdat_pos['Monoisotopic_Mass_ready'] + 1.007825

# Select relevant columns for inquiry
peak_area_inquiry = pos_peak_area_200[['Average Rt(min)', 'Average Mz'] + [col for col in pos_peak_area_200.columns if col.startswith('BH')]]

# Function to calculate ppm difference
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

ppm_threshold = 5
matched_rows_pos = []
matched_summary = []

# MS1 search and matching
for _, chem_row in tqdm(targetmzdat_pos.iterrows(), total=targetmzdat_pos.shape[0], desc="Matching compounds"):
    mw = chem_row['mz']
    matches = peak_area_inquiry[peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold)]
    if not matches.empty:
        # Append the matches along with the corresponding compound information
        for _, match_row in matches.iterrows():
            matched_rows_pos.append({
                **match_row,
                'Matched Compound': chem_row['SMILES'],
                'PREFERRED NAME': chem_row.get('PREFERRED NAME', None),
                'DTXSID_Hits': chem_row.get('DTXSID', None),
                'SMILES_STD': chem_row.get('SMILES_ready', None),
                'DTXSID': chem_row.get('DTXSID', None),
                'Name': chem_row.get('NAME', None),
                'Preferred Name': chem_row.get('PREFERRED NAME', None),
                'InChiKey_origin': chem_row.get('INCHIKEY', None)
            })
        # Record summary statistics for each targeted mz, preserving multiple hits
        matched_summary.append({
            'Targeted MZ': mw,
            'Matched Count': len(matches),
            'Average Rt(min)': matches['Average Rt(min)'].tolist(),
            'Average Mz': matches['Average Mz'].tolist(),
            'Mean Value': matches.iloc[:, 3:].mean(axis=1).tolist(),
            'Max Value': matches.iloc[:, 3:].max(axis=1).tolist(),
            'Min Value': matches.iloc[:, 3:].min(axis=1).tolist(),
            'Std Dev': matches.iloc[:, 3:].std(axis=1).tolist(),
            'RSD (%)': ((matches.iloc[:, 3:].std(axis=1) / matches.iloc[:, 3:].mean(axis=1)) * 100).tolist(),
            'SMILES_STD': chem_row.get('SMILES_ready', None),
            'DTXSID': chem_row.get('DTXSID', None),
            'CASRN': chem_row.get('CASRN', None),
            'Compound Name': chem_row.get('PREFERRED_NAME', None),
            'QC_NOTES': chem_row.get('QC_NOTES', None),
            'InChiKey_origin': chem_row.get('INCHIKEY', None)
        })

# Create a DataFrame for matched MSDIAL result rows
matched_peak_area_df_pos = pd.DataFrame(matched_rows_pos)
print(matched_peak_area_df_pos.shape)
# Create a DataFrame for matched summary statistics
matched_summary_df_pos = pd.DataFrame(matched_summary)
matched_summary_df_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/mathced_targetedmz_summary_forallsamples_pos.csv")
print(matched_summary_df_pos.shape)

# Step 3: Handling technical duplicates and calculating mean values
bh_columns = [col for col in matched_peak_area_df_pos.columns if col.startswith('BH')]
mean_columns = {}
for col in bh_columns:
    base_col = col.split('_')[0]
    if base_col not in mean_columns:
        duplicate_cols = [c for c in bh_columns if c.startswith(base_col)]
        mean_columns[base_col] = matched_peak_area_df_pos[duplicate_cols].mean(axis=1)

# Retain only the columns with calculated mean values
matched_peak_area_df_pos = pd.concat([matched_peak_area_df_pos.drop(columns=bh_columns), pd.DataFrame(mean_columns)], axis=1)

# Step 4: Retain columns with median values > 5000, filter non-numeric values, and perform DataFrame transpose
filtered_df = matched_peak_area_df_pos.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN
filtered_df = filtered_df.loc[:, (filtered_df.median(axis=0) > 5000)].dropna(how='all', axis=1)  # Retain columns with median value > 5000

# Count the number of retained columns
num_retained_columns = filtered_df.shape[1]
print(f"Number of columns retained after filtering: {num_retained_columns}")

# Calculate summary statistics for each sample column
summary_stats = {
    'Sum': filtered_df.sum(),
    'Median': filtered_df.median(),
    'Mean': filtered_df.mean(),
    'Max': filtered_df.max(),
    'Min': filtered_df.min(),
    '95th Percentile': filtered_df.quantile(0.95),
    '5th Percentile': filtered_df.quantile(0.05)
}

# Create a DataFrame from the summary statistics
summary_stats_df_pos = pd.DataFrame(summary_stats)
summary_stats_df_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/mathced_peaks_summary_forallsamples_pos.csv")

# Transpose the summary statistics DataFrame
# transposed_summary_df = summary_stats_df_pos.transpose()
# Display the final DataFrame
# print(summary_stats_df_pos)

C:\Users\yangj\AppData\Local\Temp\ipykernel_40708\443640814.py:8: DtypeWarning: Columns (18,19,20,23,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic.csv')
Matching compounds: 100%|██████████| 5144/5144 [00:47<00:00, 107.46it/s]


(3974, 410)
(1121, 15)
Number of columns retained after filtering: 104


In [14]:
targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic_withRTprediction.csv')
targetmzdat_pos['mz'] = targetmzdat_pos['Monoisotopic_Mass_ready'] + 1.007825
targetmzdat_pos.describe()

C:\Users\yangj\AppData\Local\Temp\ipykernel_40708\1418170742.py:1: DtypeWarning: Columns (18,19,20,23,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic_withRTprediction.csv')


,MONOISOTOPIC_MASS,QC_LEVEL,AVERAGE_MASS,DATA_SOURCES,BIOCONCENTRATION_FACTOR_TEST_PRED,BOILING_POINT_DEGC_TEST_PRED,48HR_DAPHNIA_LC50_MOL/L_TEST_PRED,DENSITY_G/CM^3_TEST_PRED,DEVTOX_TEST_PRED,96HR_FATHEAD_MINNOW_MOL/L_TEST_PRED,...,ring:polycycle_tricyclo_[3.5.5]_cyclopropa[cd]pentalene,ring:polycycle_tricyclo_[3.7.7]bullvalene,ring:polycycle_tricyclo_[3.7.7]semibullvalene,ring:polycycle_tricyclo_adamantane,ring:polycycle_tricyclo_benzvalene,Monoisotopic_Mass_ready,Predicted RT,Lower Bound PI,Upper Bound PI,mz
count,5144.000000,5144.000000,5144.000000,5144.000000,2934.000000,3257.000000,2.961000e+03,3615.000000,3291.000000,2.955000e+03,...,5141.0,5141.0,5141.0,5141.000000,5141.0,5144.000000,5144.000000,5144.000000,5144.000000,5144.000000
mean,352.593668,1.581843,353.298966,34.491252,3763.540278,318.394341,8.408364e-04,1.233786,0.610721,2.808877e-03,...,0.0,0.0,0.0,0.000584,0.0,330.092001,8.974895,6.877424,11.040215,331.099826
std,225.291992,1.034834,225.835608,28.511309,26376.480522,127.955324,4.578663e-03,0.392845,0.234293,2.179881e-02,...,0.0,0.0,0.0,0.024152,0.0,190.462054,3.163418,2.740376,3.340986,190.462054
min,100.013599,1.000000,100.040000,1.000000,0.001435,-41.285000,3.170000e-11,0.660000,-0.826000,5.000000e-11,...,0.0,0.0,0.0,0.000000,0.0,100.013599,0.469752,-0.526474,1.796244,101.021424
25%,187.112081,1.000000,187.360250,13.000000,4.666630,222.600000,1.390000e-06,0.967000,0.447000,8.240000e-07,...,0.0,0.0,0.0,0.000000,0.0,180.055306,6.732799,4.810620,8.766467,181.063131
50%,291.136066,1.000000,291.997000,26.000000,21.503050,309.275000,1.490000e-05,1.144000,0.614000,1.540000e-05,...,0.0,0.0,0.0,0.000000,0.0,282.292266,8.941288,6.927146,11.051048,283.300091
75%,446.351928,2.000000,446.678250,50.000000,130.542000,397.448000,1.358310e-04,1.393000,0.768500,2.264640e-04,...,0.0,0.0,0.0,0.000000,0.0,420.310921,10.712518,8.639353,12.938913,421.318746
max,2386.580035,5.000000,2404.900000,164.000000,476431.000000,848.696000,9.268300e-02,4.016000,1.446000,6.698850e-01,...,0.0,0.0,0.0,1.000000,0.0,999.141384,19.712472,15.133283,34.067956,1000.149209


In [17]:
##apply predicted RT
import pandas as pd
from tqdm import tqdm

# Load peak area data for positive ion mode after batch correction
pos_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/filtered_peak_area_for_bc_70.csv")

# Load target mz data
# targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic.csv')
targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic_withRTprediction.csv')
# import new target mz with predicted RT from Retip software


targetmzdat_pos['mz'] = targetmzdat_pos['Monoisotopic_Mass_ready'] + 1.007825

# Select relevant columns for inquiry
peak_area_inquiry = pos_peak_area_200[['Average Rt(min)', 'Average Mz'] + [col for col in pos_peak_area_200.columns if col.startswith('BH')]]

# Function to calculate ppm difference
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

def rt_diff(value1, valu2):
    return (value1-valu2)

ppm_threshold = 5
matched_rows_pos = []
matched_summary = []

# MS1 search and matching
for _, chem_row in tqdm(targetmzdat_pos.iterrows(), total=targetmzdat_pos.shape[0], desc="Matching compounds"):
    mw = chem_row['mz']
    rtlow = chem_row['Lower Bound PI']
    rthigh = chem_row['Upper Bound PI']
    matches = peak_area_inquiry[peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold)]
    #print(f"Matches after ppm filtering: {matches.shape}")  # Debugging
    if matches.empty:
        continue
    matches = matches[matches['Average Rt(min)'].apply(lambda rt: rt_diff(rt, rtlow) >=0)]
    #print(f"Matches after Rt lower filter:: {matches.shape}")  # Debugging
    if matches.empty:
        continue
    matches = matches[matches['Average Rt(min)'].apply(lambda rt: rt_diff(rt, rthigh) <=0)]
    #print(f"Matches after Rt high filter:: {matches.shape}")  # Debugging
    if matches.empty:
        continue

    if not matches.empty:
        # Append the matches along with the corresponding compound information
        for _, match_row in matches.iterrows():
            matched_rows_pos.append({
                **match_row,
                'Matched Compound': chem_row['SMILES'],
                'PREFERRED NAME': chem_row.get('PREFERRED NAME', None),
                'DTXSID_Hits': chem_row.get('DTXSID', None),
                'SMILES_STD': chem_row.get('SMILES_ready', None),
                'DTXSID': chem_row.get('DTXSID', None),
                'Name': chem_row.get('NAME', None),
                'Preferred Name': chem_row.get('PREFERRED NAME', None),
                'InChiKey_origin': chem_row.get('INCHIKEY', None)
            })
        # Record summary statistics for each targeted mz, preserving multiple hits
        matched_summary.append({
            'Targeted MZ': mw,
            'Matched Count': len(matches),
            'Average Rt(min)': matches['Average Rt(min)'].tolist(),
            'Average Mz': matches['Average Mz'].tolist(),
            'Mean Value': matches.iloc[:, 3:].mean(axis=1).tolist(),
            'Max Value': matches.iloc[:, 3:].max(axis=1).tolist(),
            'Min Value': matches.iloc[:, 3:].min(axis=1).tolist(),
            'Std Dev': matches.iloc[:, 3:].std(axis=1).tolist(),
            'RSD (%)': ((matches.iloc[:, 3:].std(axis=1) / matches.iloc[:, 3:].mean(axis=1)) * 100).tolist(),
            'SMILES_STD': chem_row.get('SMILES_ready', None),
            'DTXSID': chem_row.get('DTXSID', None),
            'CASRN': chem_row.get('CASRN', None),
            'Compound Name': chem_row.get('PREFERRED_NAME', None),
            'QC_NOTES': chem_row.get('QC_NOTES', None),
            'InChiKey_origin': chem_row.get('INCHIKEY', None)
        })

# Create a DataFrame for matched MSDIAL result rows
matched_peak_area_df_pos = pd.DataFrame(matched_rows_pos)
print(matched_peak_area_df_pos.shape)
# Create a DataFrame for matched summary statistics
matched_summary_df_pos = pd.DataFrame(matched_summary)
matched_summary_df_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/mathced_targetedmz_summary_forallsamples_pos.csv")
print(matched_summary_df_pos.shape)

# Step 3: Handling technical duplicates and calculating mean values
bh_columns = [col for col in matched_peak_area_df_pos.columns if col.startswith('BH')]
mean_columns = {}
for col in bh_columns:
    base_col = col.split('_')[0]
    if base_col not in mean_columns:
        duplicate_cols = [c for c in bh_columns if c.startswith(base_col)]
        mean_columns[base_col] = matched_peak_area_df_pos[duplicate_cols].mean(axis=1)

# Retain only the columns with calculated mean values
matched_peak_area_df_pos = pd.concat([matched_peak_area_df_pos.drop(columns=bh_columns), pd.DataFrame(mean_columns)], axis=1)

# Step 4: Retain columns with median values > 5000, filter non-numeric values, and perform DataFrame transpose
filtered_df = matched_peak_area_df_pos.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN
filtered_df = filtered_df.loc[:, (filtered_df.median(axis=0) > 5000)].dropna(how='all', axis=1)  # Retain columns with median value > 5000

# Count the number of retained columns
num_retained_columns = filtered_df.shape[1]
print(f"Number of columns retained after filtering: {num_retained_columns}")

# Calculate summary statistics for each sample column
summary_stats = {
    'Sum': filtered_df.sum(),
    'Median': filtered_df.median(),
    'Mean': filtered_df.mean(),
    'Max': filtered_df.max(),
    'Min': filtered_df.min(),
    '95th Percentile': filtered_df.quantile(0.95),
    '5th Percentile': filtered_df.quantile(0.05)
}

# Create a DataFrame from the summary statistics
summary_stats_df_pos = pd.DataFrame(summary_stats)
summary_stats_df_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/mathced_peaks_summary_forallsamples_pos.csv")

# Transpose the summary statistics DataFrame
# transposed_summary_df = summary_stats_df_pos.transpose()
# Display the final DataFrame
# print(summary_stats_df_pos)

C:\Users\yangj\AppData\Local\Temp\ipykernel_40708\3522178965.py:10: DtypeWarning: Columns (18,19,20,23,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  targetmzdat_pos = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic_withRTprediction.csv')
Matching compounds: 100%|██████████| 5144/5144 [00:41<00:00, 124.92it/s]


(1340, 410)
(535, 15)
Number of columns retained after filtering: 82


In [27]:
import pandas as pd
from tqdm import tqdm

# Load peak area data for positive ion mode after batch correction
# pos_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/filtered_peak_area_after_bc.csv")
# neg_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/filtered_peak_area_after_bc_70.csv")
neg_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/filtered_peak_area_for_bc_70.csv")
# Load target mz data
targetmzdat_neg = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic.csv')
targetmzdat_neg['mz'] = targetmzdat_neg['MONOISOTOPIC_MASS'] - 1.007825

# Select relevant columns for inquiry
peak_area_inquiry = neg_peak_area_200[['Average Rt(min)', 'Average Mz'] + [col for col in neg_peak_area_200.columns if col.startswith('BH')]]

# Function to calculate ppm difference
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

ppm_threshold = 5
matched_rows_neg = []
matched_summary = []

# MS1 search and matching
for _, chem_row in tqdm(targetmzdat_pos.iterrows(), total=targetmzdat_pos.shape[0], desc="Matching compounds"):
    mw = chem_row['mz']
    rtlow = chem_row['Lower Bound PI']
    rthigh = chem_row['Upper Bound PI']
    matches = peak_area_inquiry[peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold)]
    #print(f"Matches after ppm filtering: {matches.shape}")  # Debugging
    
    if not matches.empty:
        # Append the matches along with the corresponding compound information
        for _, match_row in matches.iterrows():
            matched_rows_neg.append({
                **match_row,
                'Matched Compound': chem_row['SMILES'],
                'PREFERRED NAME': chem_row.get('PREFERRED NAME', None),
                'DTXSID_Hits': chem_row.get('DTXSID', None),
                'SMILES_STD': chem_row.get('SMILES_ready', None),
                'DTXSID': chem_row.get('DTXSID', None),
                'Name': chem_row.get('NAME', None),
                'Preferred Name': chem_row.get('PREFERRED NAME', None),
                'InChiKey_origin': chem_row.get('INCHIKEY', None)
            })
        # Record summary statistics for each targeted mz, preserving multiple hits
        matched_summary.append({
            'Targeted MZ': mw,
            'Matched Count': len(matches),
            'Average Rt(min)': matches['Average Rt(min)'].tolist(),
            'Average Mz': matches['Average Mz'].tolist(),
            'Mean Value': matches.iloc[:, 3:].mean(axis=1).tolist(),
            'Max Value': matches.iloc[:, 3:].max(axis=1).tolist(),
            'Min Value': matches.iloc[:, 3:].min(axis=1).tolist(),
            'Std Dev': matches.iloc[:, 3:].std(axis=1).tolist(),
            'RSD (%)': ((matches.iloc[:, 3:].std(axis=1) / matches.iloc[:, 3:].mean(axis=1)) * 100).tolist(),
            'SMILES_STD': chem_row.get('SMILES_ready', None),
            'DTXSID': chem_row.get('DTXSID', None),
            'CASRN': chem_row.get('CASRN', None),
            'Compound Name': chem_row.get('PREFERRED_NAME', None),
            'QC_NOTES': chem_row.get('QC_NOTES', None),
            'InChiKey_origin': chem_row.get('INCHIKEY', None)
        })

# Create a DataFrame for matched MSDIAL result rows
matched_peak_area_df_neg = pd.DataFrame(matched_rows_neg)
print(matched_peak_area_df_neg.shape)

# Create a DataFrame for matched summary statistics
matched_summary_df_neg = pd.DataFrame(matched_summary)
print(f"#of targetedmz with hits:{matched_summary_df_neg.shape}")
# matched_summary_df_neg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/mathced_targetedmz_summary_forallsamples_Neg.csv")
print(matched_summary_df_neg.shape)

# Step 3: Handling technical duplicates and calculating mean values
bh_columns = [col for col in matched_peak_area_df_neg.columns if col.startswith('BH')]
mean_columns = {}
for col in bh_columns:
    base_col = col.split('_')[0]
    if base_col not in mean_columns:
        duplicate_cols = [c for c in bh_columns if c.startswith(base_col)]
        mean_columns[base_col] = matched_peak_area_df_neg[duplicate_cols].mean(axis=1)

# Retain only the columns with calculated mean values
matched_peak_area_df_neg = pd.concat([matched_peak_area_df_neg.drop(columns=bh_columns), pd.DataFrame(mean_columns)], axis=1)


# Step 4: Retain columns with median values > 5000, filter non-numeric values, and perform DataFrame transpose
filtered_df = matched_peak_area_df_neg.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN
filtered_df = filtered_df.loc[:, (filtered_df.median(axis=0) > 4000)].dropna(how='all', axis=1)  # Retain columns with median value > 5000

# Count the number of retained columns
num_retained_columns = filtered_df.shape[1]
print(f"Number of columns retained after filtering: {num_retained_columns}")

# Calculate summary statistics for each sample column for each individual samples
#
summary_stats = {
    'Sum': filtered_df.sum(),
    'Median': filtered_df.median(),
    'Mean': filtered_df.mean(),
    'Max': filtered_df.max(),
    'Min': filtered_df.min(),
    '95th Percentile': filtered_df.quantile(0.95),
    '5th Percentile': filtered_df.quantile(0.05)
}

# Create a DataFrame from the summary statistics
summary_stats_df_neg = pd.DataFrame(summary_stats)
# summary_stats_df_neg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/mathced_peaks_summary_forallsamples_Neg.csv")

# Transpose the summary statistics DataFrame
# transposed_summary_df_neg = summary_stats_df_neg.transpose()
# Display the final DataFrame
# print(summary_stats_df_neg)

C:\Users\yangj\AppData\Local\Temp\ipykernel_40708\1049103061.py:9: DtypeWarning: Columns (18,19,20,23,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  targetmzdat_neg = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic.csv')
Matching compounds: 100%|██████████| 5144/5144 [00:47<00:00, 108.93it/s]


(6393, 410)
#of targetedmz with hits:(1224, 15)
(1224, 15)
Number of columns retained after filtering: 159


In [25]:
import pandas as pd
from tqdm import tqdm

# Load peak area data for positive ion mode after batch correction
# pos_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/filtered_peak_area_after_bc.csv")
# neg_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/filtered_peak_area_after_bc_70.csv")
neg_peak_area_200 = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/filtered_peak_area_for_bc_70.csv")
# Load target mz data
targetmzdat_neg = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic.csv')
targetmzdat_neg['mz'] = targetmzdat_neg['MONOISOTOPIC_MASS'] - 1.007825

# Select relevant columns for inquiry
peak_area_inquiry = neg_peak_area_200[['Average Rt(min)', 'Average Mz'] + [col for col in neg_peak_area_200.columns if col.startswith('BH')]]

# Function to calculate ppm difference
def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

def rt_diff(value1, valu2):
    return (value1-valu2)

ppm_threshold = 5
matched_rows_neg = []
matched_summary = []

# MS1 search and matching
for _, chem_row in tqdm(targetmzdat_pos.iterrows(), total=targetmzdat_pos.shape[0], desc="Matching compounds"):
    mw = chem_row['mz']
    rtlow = chem_row['Lower Bound PI']
    rthigh = chem_row['Upper Bound PI']
    matches = peak_area_inquiry[peak_area_inquiry['Average Mz'].apply(lambda mz: ppm_difference(mz, mw) <= ppm_threshold)]
    #print(f"Matches after ppm filtering: {matches.shape}")  # Debugging
    if matches.empty:
        continue
    matches = matches[matches['Average Rt(min)'].apply(lambda rt: rt_diff(rt, rtlow) >=0)]
    #print(f"Matches after Rt lower filter:: {matches.shape}")  # Debugging
    if matches.empty:
        continue
    matches = matches[matches['Average Rt(min)'].apply(lambda rt: rt_diff(rt, rthigh) <=0)]
    #print(f"Matches after Rt high filter:: {matches.shape}")  # Debugging
    if matches.empty:
        continue
    
    if not matches.empty:
        # Append the matches along with the corresponding compound information
        for _, match_row in matches.iterrows():
            matched_rows_neg.append({
                **match_row,
                'Matched Compound': chem_row['SMILES'],
                'PREFERRED NAME': chem_row.get('PREFERRED NAME', None),
                'DTXSID_Hits': chem_row.get('DTXSID', None),
                'SMILES_STD': chem_row.get('SMILES_ready', None),
                'DTXSID': chem_row.get('DTXSID', None),
                'Name': chem_row.get('NAME', None),
                'Preferred Name': chem_row.get('PREFERRED NAME', None),
                'InChiKey_origin': chem_row.get('INCHIKEY', None)
            })
        # Record summary statistics for each targeted mz, preserving multiple hits
        matched_summary.append({
            'Targeted MZ': mw,
            'Matched Count': len(matches),
            'Average Rt(min)': matches['Average Rt(min)'].tolist(),
            'Average Mz': matches['Average Mz'].tolist(),
            'Mean Value': matches.iloc[:, 3:].mean(axis=1).tolist(),
            'Max Value': matches.iloc[:, 3:].max(axis=1).tolist(),
            'Min Value': matches.iloc[:, 3:].min(axis=1).tolist(),
            'Std Dev': matches.iloc[:, 3:].std(axis=1).tolist(),
            'RSD (%)': ((matches.iloc[:, 3:].std(axis=1) / matches.iloc[:, 3:].mean(axis=1)) * 100).tolist(),
            'SMILES_STD': chem_row.get('SMILES_ready', None),
            'DTXSID': chem_row.get('DTXSID', None),
            'CASRN': chem_row.get('CASRN', None),
            'Compound Name': chem_row.get('PREFERRED_NAME', None),
            'QC_NOTES': chem_row.get('QC_NOTES', None),
            'InChiKey_origin': chem_row.get('INCHIKEY', None)
        })

# Create a DataFrame for matched MSDIAL result rows
matched_peak_area_df_neg = pd.DataFrame(matched_rows_neg)
print(matched_peak_area_df_neg.shape)

# Create a DataFrame for matched summary statistics
matched_summary_df_neg = pd.DataFrame(matched_summary)
print(f"#of targetedmz with hits:{matched_summary_df_neg.shape}")
matched_summary_df_neg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/mathced_targetedmz_summary_forallsamples_Neg.csv")
print(matched_summary_df_neg.shape)

# Step 3: Handling technical duplicates and calculating mean values
bh_columns = [col for col in matched_peak_area_df_neg.columns if col.startswith('BH')]
mean_columns = {}
for col in bh_columns:
    base_col = col.split('_')[0]
    if base_col not in mean_columns:
        duplicate_cols = [c for c in bh_columns if c.startswith(base_col)]
        mean_columns[base_col] = matched_peak_area_df_neg[duplicate_cols].mean(axis=1)

# Retain only the columns with calculated mean values
matched_peak_area_df_neg = pd.concat([matched_peak_area_df_neg.drop(columns=bh_columns), pd.DataFrame(mean_columns)], axis=1)

# Step 4: Retain columns with median values > 5000, filter non-numeric values, and perform DataFrame transpose
filtered_df = matched_peak_area_df_neg.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN
filtered_df = filtered_df.loc[:, (filtered_df.median(axis=0) > 4000)].dropna(how='all', axis=1)  # Retain columns with median value > 5000

# Count the number of retained columns
num_retained_columns = filtered_df.shape[1]
print(f"Number of columns retained after filtering: {num_retained_columns}")

# Calculate summary statistics for each sample column for each individual samples
#
summary_stats = {
    'Sum': filtered_df.sum(),
    'Median': filtered_df.median(),
    'Mean': filtered_df.mean(),
    'Max': filtered_df.max(),
    'Min': filtered_df.min(),
    '95th Percentile': filtered_df.quantile(0.95),
    '5th Percentile': filtered_df.quantile(0.05)
}

# Create a DataFrame from the summary statistics
summary_stats_df_neg = pd.DataFrame(summary_stats)
summary_stats_df_neg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/mathced_peaks_summary_forallsamples_Neg.csv")

# Transpose the summary statistics DataFrame
# transposed_summary_df_neg = summary_stats_df_neg.transpose()
# Display the final DataFrame
# print(summary_stats_df_neg)

C:\Users\yangj\AppData\Local\Temp\ipykernel_40708\3932377325.py:9: DtypeWarning: Columns (18,19,20,23,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  targetmzdat_neg = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Plastic_map_chemicallist_match/Plastic_Chemical_List_organic.csv')
Matching compounds: 100%|██████████| 5144/5144 [00:39<00:00, 131.27it/s]


(2599, 410)
#of targetedmz with hits:(670, 15)
(670, 15)
Number of columns retained after filtering: 155


In [18]:
from rdkit import Chem
from rdkit.Chem import MolToSmiles
##cross refrence, cross check with blood exposome database
bloodexpo_dat = pd.read_csv("D:/UCSF_postdoc_topic/REVEAL_topics/references/blood_exposome_database/blood_exposome_chemicals_july_2023.csv")

# Step 1: Filter rows with available MW (MONOISOTOPIC MASS) between 100 and 1000 and classified as organic
bloodexpo_modified = []
for index, row in bloodexpo_dat.iterrows():
    smiles = row['CanonicalSMILES']  # Assuming the SMILES column is named 'SMILES'
    mol = Chem.MolFromSmiles(smiles)
    canonical_smiles = Chem.MolToSmiles(mol, canonical=True) if mol else None
    bloodexpo_modified.append({
        **row,
        "SMILES_ready": canonical_smiles})

# Create a new DataFrame from the filtered rows
bloodexpo_modified = pd.DataFrame(bloodexpo_modified)


[13:49:09] WARNING: not removing hydrogen atom without neighbors
[13:49:10] Explicit valence for atom # 5 Cl, 3, is greater than permitted
[13:49:10] Explicit valence for atom # 4 Cl, 3, is greater than permitted
[13:49:12] Explicit valence for atom # 7 Br, 3, is greater than permitted
[13:49:12] Explicit valence for atom # 20 Cl, 3, is greater than permitted
[13:49:13] Explicit valence for atom # 6 Cl, 5, is greater than permitted
[13:49:14] Explicit valence for atom # 3 Cl, 3, is greater than permitted
[13:49:15] Explicit valence for atom # 5 Br, 3, is greater than permitted
[13:49:16] Explicit valence for atom # 9 Cl, 3, is greater than permitted
[13:49:22] WARNING: not removing hydrogen atom without neighbors


In [26]:
# Match SMILES from match_summary_df with bloodexpo_modified and retain hits
matched_hits = pd.merge(matched_summary_df_neg, bloodexpo_modified, how='inner', left_on='SMILES_STD', right_on='SMILES_ready')

# Add matched titles to the original DataFrame
title_dict = matched_hits.groupby('SMILES_STD')['Title'].apply(list).to_dict()
matched_summary_df_neg['matched_with_bloodexpo'] = matched_summary_df_neg['SMILES_STD'].map(title_dict).apply(lambda x: x if isinstance(x, list) else [])

# Display the updated DataFrame
matched_summary_df_neg.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/mathced_peaks_summary_withbloodexpo_Neg.csv")

##annotate the targeted mz with the function information and the production volume information from the EST paper
plasticmap_est = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/plastic_related_chemicals/plasticmap_from_ESTpaper.csv',encoding='ISO-8859-1')
# Match Inchikey from match_summary_df with bloodexpo_modified and retain hits
matched_summary_df_neg_plastic_note = pd.merge(matched_summary_df_neg, plasticmap_est, how='inner', left_on='InChiKey_origin', right_on='InChI_key')
matched_summary_df_neg_plastic_note.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Neg_AlignmentResults/mathced_peaks_summary_withplasticannotation_Neg.csv")

In [ ]:
# Match SMILES from match_summary_df with bloodexpo_modified and retain hits
matched_hits = pd.merge(matched_summary_df_pos, bloodexpo_modified, how='inner', left_on='SMILES_STD', right_on='SMILES_ready')

# Add matched titles to the original DataFrame
title_dict = matched_hits.groupby('SMILES_STD')['Title'].apply(list).to_dict()
matched_summary_df_pos['matched_with_bloodexpo'] = matched_summary_df_pos['SMILES_STD'].map(title_dict).apply(lambda x: x if isinstance(x, list) else [])

# Display the updated DataFrame
matched_summary_df_pos.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/mathced_peaks_summary_withbloodexpo_pos.csv")

##annotate the targeted mz with the function information and the production volume information from the EST paper
plasticmap_est = pd.read_csv('D:/UCSF_postdoc_topic/REVEAL_topics/plastic_related_chemicals/plasticmap_from_ESTpaper.csv',encoding='ISO-8859-1')
# Match Inchikey from match_summary_df with bloodexpo_modified and retain hits
matched_summary_df_pos_plastic_note = pd.merge(matched_summary_df_pos, plasticmap_est, how='inner', left_on='InChiKey_origin', right_on='InChI_key')
matched_summary_df_pos_plastic_note.to_csv("D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/Pos_AlignmentResults/mathced_peaks_summary_withplasticannotation_pos.csv")


In [ ]:
#generate the text for CFMID prediction for positive and negative
#cross check with other databases for experimental LCMSMS database

In [29]:
import pickle

# Save all variables in the environment, excluding un-picklable ones
with open('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/scripts/saved_environment.pkl', 'wb') as f:
    save_dict = {}
    for var_name, var_value in globals().items():
        if not var_name.startswith('__') and var_name != 'f':
            try:
                pickle.dumps(var_value)
                save_dict[var_name] = var_value
            except (pickle.PicklingError, AttributeError, TypeError):
                # Ignore un-picklable variables
                print(f"Variable '{var_name}' could not be pickled and was ignored.")
    pickle.dump(save_dict, f)

print("Environment saved successfully.")


Variable 'get_ipython' could not be pickled and was ignored.
Variable 'exit' could not be pickled and was ignored.
Variable 'quit' could not be pickled and was ignored.
Variable 'open' could not be pickled and was ignored.
Variable 'sys' could not be pickled and was ignored.
Variable 'os' could not be pickled and was ignored.
Variable 'np' could not be pickled and was ignored.
Variable 'stats' could not be pickled and was ignored.
Variable 'pd' could not be pickled and was ignored.
Variable 'plt' could not be pickled and was ignored.
Variable 'sb' could not be pickled and was ignored.
Variable 'rd' could not be pickled and was ignored.
Variable 'matplotlib' could not be pickled and was ignored.
Variable 'warnings' could not be pickled and was ignored.
Variable 'Chem' could not be pickled and was ignored.
Variable 'MolToSmiles' could not be pickled and was ignored.
Variable 'pickle' could not be pickled and was ignored.
Environment saved successfully.


In [ ]:
import pickle
import os
import pandas as pd
import rdkit 
from tqdm import tqdm


def ppm_difference(value1, value2):
    return abs(value1 - value2) / value1 * 1e6

# Load all variables from the saved environment
with open('D:/UCSF_postdoc_topic/REVEAL_topics/REVEAL_200samples_analysis/saved_environment.pkl', 'rb') as f:
    loaded_env = pickle.load(f)

# Assign variables back to the global namespace
globals().update(loaded_env)

print("Environment loaded successfully.")

Environment loaded successfully.
